In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ffabious_innodorm_path = kagglehub.dataset_download('ffabious/innodorm')

print('Data source import complete.')


100%|██████████| 71.2M/71.2M [00:00<00:00, 272MB/s]

Extracting files...


Data source import complete.


In [9]:
print(ffabious_innodorm_path)

/root/.cache/kagglehub/datasets/ffabious/innodorm/versions/1


# YOLO Fine-tuning for Furniture Detection (InnoDorm Dataset)

This notebook demonstrates how to fine-tune a YOLO model on the custom "InnoDorm" dataset labeled with Label Studio.

## Prerequisites
1.  **Label Data**: Label your images in Label Studio.
2.  **Export**: Export the project as **YOLO**.
3.  **Setup**: Unzip the export into `data/innodorm`. The structure should look like:
    ```
    data/innodorm/
        images/
        labels/
        classes.txt
        notes.json
    ```
    (Or if Label Studio exported with train/val split, that's fine too. We will handle the split below.)

## Steps:
1.  **Setup**: Install `ultralytics`.
2.  **Data Preparation**:
    *   Verify the dataset structure in `data/innodorm`.
    *   Split into `train` and `val` sets if not already done.
    *   Create `dataset.yaml`.
3.  **Training**:
    *   Load a pre-trained YOLO model (e.g., `yolo11n.pt`).
    *   Fine-tune the model on the InnoDorm dataset.

In [2]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.6 MB/s eta 0:00:00


In [3]:
!cp -R /kaggle/input/innodorm /kaggle/working/input/innodorm

cp: cannot stat '/kaggle/input/innodorm': No such file or directory


In [10]:
import os
import shutil
import yaml
import random
from pathlib import Path
from ultralytics import YOLO

# Define paths
INPUT_ROOT = Path(ffabious_innodorm_path).resolve()
OUTPUT_ROOT = Path("/kaggle/working/output").resolve()
IMAGES_DIR = INPUT_ROOT / "images"
LABELS_DIR = INPUT_ROOT / "labels"

print(f"Dataset root: {INPUT_ROOT}")


Dataset root: /root/.cache/kagglehub/datasets/ffabious/innodorm/versions/1


In [11]:
def train_val_split(items, val_ratio=0.2, seed=42):
    import random

    items = list(items)
    random.seed(seed)
    random.shuffle(items)

    n_val = int(len(items) * val_ratio)
    return items[n_val:], items[:n_val]

In [12]:
# Organize dataset into train/val split if not already done
# Label Studio YOLO export usually puts all images in 'images' and labels in 'labels'
# We need:
#   images/train, images/val
#   labels/train, labels/val

def organize_dataset(root_path, val_size=0.2):
    root = Path(root_path)
    images_path = root / "images"
    labels_path = root / "labels"

    # Check if already split
    if (images_path / "train").exists():
        print("Dataset already appears to be split.")
        return

    print("Organizing dataset into train/val split...")

    # Get all image files
    all_images = list(images_path.glob("*.jpg")) + list(images_path.glob("*.png")) + list(images_path.glob("*.jpeg"))

    if not all_images:
        print("No images found! Please check if you unzipped the dataset correctly.")
        return

    # Split
    train_imgs, val_imgs = train_val_split(all_images, test_size=val_size, random_state=42)

    # Create directories
    (images_path / "train").mkdir(parents=True, exist_ok=True)
    (images_path / "val").mkdir(parents=True, exist_ok=True)
    (labels_path / "train").mkdir(parents=True, exist_ok=True)
    (labels_path / "val").mkdir(parents=True, exist_ok=True)

    # Move files
    def move_files(file_list, split_name):
        for img_path in file_list:
            # Move image
            shutil.move(str(img_path), str(images_path / split_name / img_path.name))

            # Move corresponding label
            label_name = img_path.stem + ".txt"
            label_src = labels_path / label_name
            if label_src.exists():
                shutil.move(str(label_src), str(labels_path / split_name / label_name))

    move_files(train_imgs, "train")
    move_files(val_imgs, "val")

    print(f"Moved {len(train_imgs)} to train and {len(val_imgs)} to val.")

organize_dataset(INPUT_ROOT)


Dataset already appears to be split.


In [13]:
# Create dataset.yaml
# We need to read classes.txt from Label Studio export to get class names
classes_file = INPUT_ROOT / "classes.txt"

if classes_file.exists():
    with open(classes_file, 'r') as f:
        class_names = [line.strip() for line in f.readlines() if line.strip()]

    # Create dictionary mapping index to name
    names_dict = {i: name for i, name in enumerate(class_names)}

    yaml_content = {
        'path': str(INPUT_ROOT),
        'train': 'images/train',
        'val': 'images/val',
        'names': names_dict
    }

    yaml_path = INPUT_ROOT / "dataset.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f)

    print(f"Created dataset.yaml at {yaml_path}")
    print("Classes:", names_dict)
else:
    print("Error: classes.txt not found. Please ensure Label Studio export included it.")


Created dataset.yaml at /root/.cache/kagglehub/datasets/ffabious/innodorm/versions/1/dataset.yaml
Classes: {0: 'bed', 1: 'book', 2: 'chair', 3: 'clock', 4: 'couch', 5: 'dining table', 6: 'keyboard', 7: 'laptop', 8: 'microwave', 9: 'mouse', 10: 'oven', 11: 'potted plant', 12: 'refrigerator', 13: 'sink', 14: 'toaster', 15: 'toilet', 16: 'tv', 17: 'vase'}


In [14]:
# Train the model
# Load the model (using the one in models/yolo/yolo11n.pt if available, else download)
model_path = "../models/yolo/yolo11n.pt"
if not os.path.exists(model_path):
    print(f"Model not found at {model_path}, using 'yolo11n.pt' (will download)")
    model_name = "yolo11n.pt"
else:
    print(f"Loading model from {model_path}")
    model_name = model_path

model = YOLO(model_name)

# Train
# We specify the data config we just created
# epochs=50 is a good start for a small custom dataset
# We enable strong data augmentation to prevent overfitting on the small dataset
results = model.train(
    data=str(INPUT_ROOT / "dataset.yaml"),
    epochs=50,
    imgsz=640,
    project=str(OUTPUT_ROOT / "yolo_training"),
    name="innodorm_finetune",
    exist_ok=True,

    # Augmentation Hyperparameters
    # These are applied on-the-fly during training
    degrees=15.0,      # Rotate images by +/- 15 degrees
    translate=0.1,     # Translate images by +/- 10%
    scale=0.5,         # Scale images by +/- 50%
    shear=2.5,         # Shear images by +/- 2.5 degrees
    perspective=0.001, # Perspective distortion (simulates different camera angles)
    flipud=0.0,        # Vertical flip (disabled as furniture doesn't usually appear upside down)
    fliplr=0.5,        # Horizontal flip (50% chance)
    mosaic=1.0,        # Mosaic augmentation (combines 4 images) - very effective
    mixup=0.1,         # Mixup augmentation (blends 2 images)
    hsv_h=0.015,       # HSV-Hue augmentation
    hsv_s=0.7,         # HSV-Saturation augmentation
    hsv_v=0.4,         # HSV-Value (Brightness) augmentation
)

print("Training complete.")

Model not found at ../models/yolo/yolo11n.pt, using 'yolo11n.pt' (will download)
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/root/.cache/kagglehub/datasets/ffabious/innodorm/versions/1/dataset.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=innodorm_fin